In [ ]:
import gc
import sys
import os
from GPLpyMaster.gplpy.gggp.grammar import CFG, ProbabilisticModel
from GPLpyMaster.gplpy.evo.evolution import Experiment, Setup, Problem, Evolution_EDA, Evolution_WX
from GPLpyMaster.gplpy.evo.log import DBLogger
from GPLpyMaster.gplpy.gggp.derivation import Derivation, WX, OnePointMutation
from GPLpyMaster.gplpy.gggp.metaderivation import MetaDerivation, EDA


from bson.objectid import ObjectId
import tensorflow

# SETUP EXAMPLES ###########################################################
EDX_setup = Setup(name='EDX', evolution=Evolution_EDA, max_recursions = 100, crossover=EDA, selection_rate=0.5, exploration_rate=0.01, model_update_rate=0.5, offspring_rate=.4)
class DFFNN(Problem):
    epochs = 2
    batch_size = 128

    @staticmethod
    def fitness(individual, args):
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Dense, Activation

        topology = list(map(len, str(individual.derivation).replace(' ','').split("0")))
        input_size, num_classes, X_train, X_test, y_train, y_test = args
        model = Sequential()
        # First layer and hidden layer
        model.add(Dense(topology.pop(0), activation='relu', input_dim=input_size))
        # Hidden layers
        for layer_size in topology:
            model.add(Dense(layer_size, activation='relu'))
        # Output layer
        model.add(Dense(1 if num_classes==2 else num_classes, activation='sigmoid' if num_classes==2 else 'softmax'))

        # Setup optimizer
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        history = model.fit(X_train, y_train,
                            epochs=DFFNN.epochs,
                            batch_size=DFFNN.batch_size,
                            verbose=0,
                            validation_data=(X_test, y_test))

        score = model.evaluate(X_test, y_test, verbose=0)
        #print('Test loss:', score[0])
        #print('Test accuracy:', score[1])
        individual._fitness = score[0]
        individual.learning_iterations = len(history.epoch)
        individual.mature.set()
        

if __name__ == "__main__":
    sys.setrecursionlimit(10000)
        
    ## IS 
    study = "DFFNN"
    study_id = None
    #study_id = ObjectId("590b99f0d140a535c9dfbe12")

    # Grammar initialization
    grammar_file = "GPLpyMaster\gr\\"+study.replace(' ', '_') + '.gr'
    gr = CFG(grammar_file)

    # logger initialization
    # Set to True to log into mongodb

    # Setup problem    
    from sklearn import datasets
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from tensorflow.keras.utils import to_categorical

    exp_name = "Cancer"
    X, y = datasets.load_breast_cancer(return_X_y=True)
    num_classes = len(set(y))
    input_size = X.shape[1]
    if num_classes > 2:
        y = to_categorical(y, num_classes)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    y_train = y_train.reshape(y_train.size, 1)
    y_test = y_test.reshape(y_test.size, 1)
    args = (input_size, num_classes, X_train, X_test, y_train, y_test)
    
    resul = []
    # Run
    for i in range(15):
        ids = Experiment(study=study, experiment=exp_name, grammar=gr, problem=DFFNN, fitness_args=args,
                    setups=[EDX_setup], logger=logger, samples = 1).run()
        resul.append(ids)
        gc.collect()

[]
Experiment: Cancer
setup_name: EDX
initialization: uniform
crossover: EDA
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Best individual fitness: 0.41, Average fitness: 0.63, Improvement: 1.00
Best individual fitness: 0.33, Average fitness: 0.56, Improvement: 0.11
Best individual fitness: 0.33, Average fitness: 0.55, Improvement: 0.03
Best individual fitness: 0.33, Average fitness: 0.52, Improvement: 0.04
Best individual fitness: 0.33, Average fitness: 0.53, Improvement: -0.01
Best individual fitness: 0.33, Average fitness: 0.50, Improvement: 0.04
Best individual fitness: 0.33, Average fitness: 0.51, Improvement: -0.01
Best individual fitness: 0.33, Average fitness: 0.48, Improvement: 0.05
